In [67]:
from pymongo import MongoClient
from datetime import datetime as dt
import pandas as pd
from math import isnan
from config import DB_HOST, DB_PASSWORD, DB_USERNAME

In [114]:
def get_score(throws, par):
    score = int(throws - par)

    if throws == 1:
        score_name = "Ace"
        return score, score_name
    
    if score == -3:
        score_name = "Albatross"
        return score, score_name

    if score == -2:
        score_name = "Eagle"
        return score, score_name

    if score == -1:
        score_name = "Birdie"
        return score, score_name

    if score == 0:
        score_name = "Par"
        return score, score_name

    if score == 1:
        score_name = "Bogey"
        return score, score_name

    if score == 2:
        score_name = "Double Bogey"
        return score, score_name

    if score == 3:
        score_name = "Triple Bogey"
        return score, score_name

    if score >= 4:
        score_name = "4+"
        return score, score_name


In [2]:
mongo = MongoClient(f"mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/disc_golf?retryWrites=true&w=majority")

In [17]:
courses = list(mongo["disc_golf"]["courses"].find({"udisc_name": {"$regex": ".*"}}))

In [59]:
df = pd.read_csv("../Resources/UdiscScorecards.csv")
df = df[df['PlayerName'] != 'Par']

In [60]:
course_round = df.merge(pd.DataFrame(courses), how="inner", left_on="CourseName", right_on="udisc_name")

In [88]:
holes = [f"Hole{h}" for h in range(19)[1:]]
melted_scores = pd.melt(df.rename(columns={"+/-":"Par"}), id_vars=["PlayerName", "CourseName", "LayoutName", "Date"], value_vars=holes, var_name="Hole", value_name="Throws")

In [89]:
melted_scores["Hole"] = [int(h.replace("Hole", "")) for h in melted_scores["Hole"]]

clean_melted_scores = melted_scores.dropna()

In [90]:
clean_melted_scores

,PlayerName,CourseName,LayoutName,Date,Hole,Throws
0,ConnorHz,Silverview Park,2017 Redesign,2021-05-31 14:56,1,4.0
1,Spenser Barton,Silverview Park,2017 Redesign,2021-05-31 14:56,1,4.0
2,ConnorHz,Hansen Park,New 14 Hole Layout,2021-05-31 13:30,1,4.0
3,Spenser Barton,Hansen Park,New 14 Hole Layout,2021-05-31 13:30,1,5.0
4,ConnorHz,Silverview Park,2017 Redesign,2021-05-30 18:28,1,3.0
...,...,...,...,...,...,...
410,ConnorHz,Acorn Park,Long Tees Long Baskets,2021-05-17 17:54,17,4.0
419,ConnorHz,Lakewood Hills,Main,2021-04-25 08:48,17,4.0
431,ConnorHz,Elm Creek Park Reserve,Main,2021-05-29 08:11,18,5.0
435,ConnorHz,Acorn Park,Long Tees Long Baskets,2021-05-17 17:54,18,5.0


In [98]:
udisc_courses = [{"CourseName": c["udisc_name"], "Layouts": c["layouts"]} for c in courses]
udisc_course_layout_holes = []
for uc in udisc_courses:
    for l in uc["Layouts"]:
        for h in l["holes"]:
            udisc_course_layout_holes.append({"CourseName": uc["CourseName"], "LayoutName": l["name"], "Hole": h['hole'], "Par": h['par'], "Distance": h['distance']})

pd.DataFrame(udisc_course_layout_holes)

,CourseName,LayoutName,Hole,Par,Distance
0,Bassett Creek Park,2015 Redesign,1,3,249
1,Bassett Creek Park,2015 Redesign,2,3,125
2,Bassett Creek Park,2015 Redesign,3,3,233
3,Bassett Creek Park,2015 Redesign,4,3,194
4,Bassett Creek Park,2015 Redesign,5,4,463
...,...,...,...,...,...
100,Silverview Park,2017 Redesign,5,3,266
101,Silverview Park,2017 Redesign,6,3,174
102,Silverview Park,2017 Redesign,7,3,190
103,Silverview Park,2017 Redesign,8,4,259


In [115]:
merged_clean_melted_scores = clean_melted_scores.merge(pd.DataFrame(udisc_course_layout_holes), on=["CourseName", "LayoutName", "Hole"], how="inner")

merged_clean_melted_scores["Score"] = pd.DataFrame(merged_clean_melted_scores.apply(lambda row: get_score(row["Throws"], row["Par"]),axis=1))

merged_clean_melted_scores[['Score', 'ScoreName']] = pd.DataFrame(merged_clean_melted_scores['Score'].tolist(), index=merged_clean_melted_scores.index)

merged_clean_melted_scores

,PlayerName,CourseName,LayoutName,Date,Hole,Throws,Par,Distance,Score,ScoreName
0,ConnorHz,Silverview Park,2017 Redesign,2021-05-31 14:56,1,4.0,3,157,1.0,Bogey
1,Spenser Barton,Silverview Park,2017 Redesign,2021-05-31 14:56,1,4.0,3,157,1.0,Bogey
2,ConnorHz,Silverview Park,2017 Redesign,2021-05-30 18:28,1,3.0,3,157,0.0,Par
3,ConnorHz,Silverview Park,2017 Redesign,2021-05-30 18:03,1,3.0,3,157,0.0,Par
4,ConnorHz,Silverview Park,2017 Redesign,2021-05-07 08:02,1,3.0,3,157,0.0,Par
...,...,...,...,...,...,...,...,...,...,...
276,ConnorHz,Acorn Park,Long Tees Long Baskets,2021-05-17 17:54,17,4.0,3,344,1.0,Bogey
277,ConnorHz,Lakewood Hills,Main,2021-04-25 08:48,17,4.0,3,344,1.0,Bogey
278,ConnorHz,Elm Creek Park Reserve,Main,2021-05-29 08:11,18,5.0,4,686,1.0,Bogey
279,ConnorHz,Acorn Park,Long Tees Long Baskets,2021-05-17 17:54,18,5.0,4,492,1.0,Bogey


In [61]:
mongo["disc_golf"]['rounds'].insert_many(df.rename(columns={"+/-":"Par"}).to_dict(orient="records"))


,PlayerName,CourseName,LayoutName,Date,Total,+/-,Hole1,Hole2,Hole3,Hole4,...,Hole9,Hole10,Hole11,Hole12,Hole13,Hole14,Hole15,Hole16,Hole17,Hole18
1,ConnorHz,Silverview Park,2017 Redesign,2021-05-31 14:56,31,1.0,4,2,3,5,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Spenser Barton,Silverview Park,2017 Redesign,2021-05-31 14:56,36,6.0,4,3,3,5,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ConnorHz,Hansen Park,New 14 Hole Layout,2021-05-31 13:30,50,8.0,4,4,4,5,...,2,4.0,3.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
5,Spenser Barton,Hansen Park,New 14 Hole Layout,2021-05-31 13:30,63,21.0,5,5,5,6,...,4,5.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN
7,ConnorHz,Silverview Park,2017 Redesign,2021-05-30 18:28,31,1.0,3,2,4,4,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ConnorHz,Silverview Park,2017 Redesign,2021-05-30 18:03,31,1.0,3,2,2,4,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,ConnorHz,Elm Creek Park Reserve,Main,2021-05-29 08:11,74,18.0,5,4,4,4,...,4,4.0,4.0,5.0,3.0,6.0,3.0,4.0,3.0,5.0
13,ConnorHz,Bassett Creek Park,2015 Redesign,2021-05-25 18:30,52,6.0,4,4,3,4,...,3,4.0,3.0,4.0,3.0,4.0,3.0,NaN,NaN,NaN
15,ConnorHz,Riverfront 13 Disc Golf Course,Blue (mixed locations),2021-05-23 09:59,47,7.0,3,4,3,4,...,3,3.0,3.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN
17,ConnorHz,Hansen Park,New 14 Hole Layout,2021-05-23 08:33,50,8.0,4,5,4,4,...,2,3.0,4.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
